基于现有文本的序列，预处理成字符的序列：每 3 个字符一个序列，最高 60 个字符。

In [12]:
import keras
import numpy as np

path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text=open(path).read().lower()

print('Corpus length:', len(text))

Corpus length: 600893


In [13]:
maxlen = 60

step = 3

sentences = []
next_chars = []

for i in range(0, len(text) - maxlen - step):
    sentences.append(text[i:i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
# 进行 one-hot 编码
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    



Number of sequences: 600830
Unique characters: 57
Vectorization...


构建一个网络：LSTM + Dense + softMax。

In [14]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape = (maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

### 采样：对后续的字符进行预测：
 1. 给定当前的文本序列，从模型中得到下一个字符的概率。
 2. 根据给定的**温度**对概率重新进行加权（温度越高随机性越强，越有趣，温度越低越通顺）
 3. 根据新概率随机采样
 4. 将获取的字符追加到当前文本后

In [15]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    # 根据概率值生成随机结果
    probas = np.random.multinomial(1, preds, 1) 
    return np.argmax(probas)

In [16]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    # 从随机位置找个种子
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index:start_index + maxlen]
    print('--- generating with seed: "', generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('--- temperature:', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)

epoch 1
4694/4694 [==============================] - 327s 70ms/step - loss: 1.7273
--- generating with seed: " hus acts the powerful, the superior, the
original state foun"
--- temperature: 0.2
hus acts the powerful, the superior, the
original state found of the discoverism of the such a something the world and the senses of the same with the present and and the same with the world of the consequences of the present and the same with the morality of the same the god the such a man which is a world of the excensitions of the sentiment of the sentiment of the same with the same with the present of the same with the comprehension of the same which i--- temperature: 0.5
esent of the same with the comprehension of the same which is the possed, so is the individually comprehension, and which self-do the constant of which
better the modernation and is a freedom as it is varied to the soul of means of the man is all the means of the decively and present at how have and and it is its regard of 

in this well great still a mear, people to -has, not preparation. the
ceaused; at the averation, and error.

7. how now upon, to old often
"formal."--at perhiate reasing nond--how inflict upeinring but
great patieepoch 5
4694/4694 [==============================] - 313s 67ms/step - loss: 1.4081
--- generating with seed: " k to the periods of crude
civilization, or if we contemplate"
--- temperature: 0.2
k to the periods of crude
civilization, or if we contemplate the subjection of the same and the same conscience, and the constitutes and the desire of the same conditions of the present to the senses and always and the same and and the salvality of the same constituted to the same the same present of the senses and the comparison the subjection of the same strong in the present to the same standard of the same the strong and the same the same spirit of the--- temperature: 0.5
d of the same the strong and the same the same spirit of the same artists that what is a soul, at the fact that 

<ipython-input-15-198c853561bf>:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


and in the fact that the soul of the soul and above all the sense of the conscientions--- temperature: 0.5
oul of the soul and above all the sense of the conscientions are understanding of him necessary probably and the labit, one may be more as it is a prichen one has the same secret in every flay here in the sense of the same failure of the sense of the sentic sense, as all the case and answert of such as a conditions, the master of such a strangely counter the end and feeling of the far belong that when it has all even the arrory in the conscientific of self--- temperature: 1.0
when it has all even the arrory in the conscientific of self do all feared" wish, and noget alled to make no . one is certain bre. but the wooks oneself x that consisted lipent table--he feeling, i
virtuelt objective looked place strong christianes, which can conclusion men, this dilen much object, as the blings,
only of the syity--of laven still
something woman thing as history, too bad brought.
a fellow i b

inspired by the impulse to self present the superhronant of the present interpretation, and a proper the superhronant of the morality of the strange of the most strange of the senses of the most superhreiting of the strange of the most the senses and the morality of the senses and the most man of the most and the standards of the most standards of the senses of the most for the world and senses and and the most not the most comprehen--- temperature: 0.5
the world and senses and and the most not the most comprehensive and the false present the soul and man the present power the man and had so that it was an age we are not at present it is the profound by the opposite and compresses and and preservation of their stranges of the interprets and the morality of meaning and living and possible. it has been at one father in the problem of man of the fact belief of the present one possession is the ancient the se--- temperature: 1.0
t belief of the present one possession is the ancient the sen

4694/4694 [==============================] - 448s 95ms/step - loss: 1.3571
--- generating with seed: " eemly attitude for the truth. christianity, it is true, had "
--- temperature: 0.2
eemly attitude for the truth. christianity, it is true, had to a such and the most proper of the fact of the masters and deceived and object and any more principle of the consciences of the conscience, and an any fact that is the most being and which is the propert to a such and and the standard of the world of the most postured to the most the most the most proper to the same to the same the ancient and and an any personal superious and such and the conce--- temperature: 0.5
ent and and an any personal superious and such and the conception of sinces and the poster,
and a condition of the most stand of the philosophers is former of the individual experiences. other and much as a purely in the labor and science of our personal end which is the sentiments and more spirituality of the more instinct, of the

knifterem. as pais ro sompcers the juralkef; o hise siss[.=--perhaptseu impensed, min rigid his ctalostrpiticio he
  vendioo and as i mind notid fin "ne
dowy
that is the
f keanting of
himsenf,m vremind
inilgedible irprimace the os y york fod tod arpatunatised, light pitseful mind womed yourulwepoch 20
4694/4694 [==============================] - 452s 96ms/step - loss: 6.1679
--- generating with seed: " whom one may talk and laugh when one is disposed to talk
and"
--- temperature: 0.2
whom one may talk and laugh when one is disposed to talk
and vhe tteuse et  oncen tsphe werenerevent.    = thusnistefe at thwacth ho al mem go ehatefy and rge padin ise. and ther, dow hto thy andf ant ham, hash os., "a the irlse af te the tan the e that imhede thc. thut wone move a =se angt brefm a qo tthc thefb inas hh" wo iefontide wene it on the thnate. 
e ine the  of le the cder. the thexes t th aqurec in0te ther ant an th ty-aals ace(dte a". the and

--- temperature: 0.5
th aqurec in0te ther ant an th

<ipython-input-15-198c853561bf>:7: RuntimeWarning: invalid value encountered in greater_equal
  probas = np.random.multinomial(1, preds, 1)


ValueError: pvals < 0, pvals > 1 or pvals contains NaNs